In [3]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import os

def fetch_dvol_data_paginated(start_date, end_date, resolution,verbose=False):
    url = "https://www.deribit.com/api/v2/public/get_volatility_index_data"
    end_time = int(datetime.strptime(end_date, "%Y-%m-%d").timestamp() * 1000)
    start_time = int(datetime.strptime(start_date, "%Y-%m-%d").timestamp() * 1000)
    
    # Convert resolution to integer if passed as string
    resolution = int(resolution)
    interval = resolution * 1000 * 1000  # Backward time window (e.g., 1000 rows)

    all_data = []
    
    while end_time > start_time:
        # Calculate the start time window for the next request
        start_window = end_time - interval
        if start_window < start_time:
            start_window = start_time
        
        params = {
            'currency': 'BTC',
            'start_timestamp': start_window,
            'end_timestamp': end_time,
            'resolution': resolution
        }
        
        try:
            response = requests.get(url, params=params)
            data = response.json()
            
             # Only print if verbose is True
            if verbose:
                print("API Response:", data)

            if 'result' in data and 'data' in data['result']:
                raw_data = data['result']['data']
                
                if not raw_data:
                    if verbose:
                        print("No more data available.")
                    break
                
                # Convert to DataFrame
                df = pd.DataFrame(raw_data, columns=['timestamp', 'open', 'high', 'low', 'close'])
                df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
                all_data.append(df)
                
                # Update end time to fetch older data
                end_time = int(df['timestamp'].min().timestamp() * 1000) - resolution * 1000
            else:
                print("Unexpected API response structure or no data returned.")
                break

        except Exception as e:
            print(f"Error fetching data: {e}")
            break

    if all_data:
        final_df = pd.concat(all_data).drop_duplicates(subset='timestamp').sort_values(by='timestamp')
        return final_df
    else:
        return None


def save_or_update_csv(df, csv_filename):
    if os.path.exists(csv_filename):
        existing_df = pd.read_csv(csv_filename)
        
        # Ensure timestamp is converted to datetime in both DataFrames
        existing_df['timestamp'] = pd.to_datetime(existing_df['timestamp'])
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        
        # Combine and deduplicate
        combined_df = pd.concat([existing_df, df]).drop_duplicates(subset='timestamp').sort_values(by='timestamp')
        combined_df.to_csv(csv_filename, index=False)
        print(f"Updated {csv_filename}")
    else:
        df.to_csv(csv_filename, index=False)
        print(f"Data saved to {csv_filename}")


In [4]:
if __name__ == "__main__":
    start_date = "2024-06-01"
    end_date = "2025-03-11"
    resolution = "3600"  # 1hr resolution 3600 sec
    
    # Fetch paginated DVOL data
    dvol_data = fetch_dvol_data_paginated(start_date, end_date, resolution, verbose=False)
    
    if dvol_data is not None:
        csv_path = r"C:\Users\USER\Documents\Tradingstuff"
        csv_filename = os.path.join(csv_path if csv_path else '.', "BTC_IV_Deribit_hour.csv")
        save_or_update_csv(dvol_data, csv_filename)



Updated C:\Users\USER\Documents\Tradingstuff\BTC_IV_Deribit_hour.csv
